# Trénovanie hlbokých sietí - optimalizácia a rady

Na minulom cvičení sme implementovali jednoduchú konvolučnú sieť a pri riešení vašich zadaní sa stretnete s ďalšími často používanými topológiami a architektúrami. Definovanie neurónovej siete je však iba prvý (a najjednoduchší) krok vývoja modelov, ktoré neskôr použijete v reálnych aplikáciách. Na to, aby ste vaše modely vedeli aj využiť, potrebujete ich vhodne natrénovať, tento proces je však skomplikovaný niekoľkými javmi. Cieľom dnešného cvičenia je oboznámiť vás s týmito problémami a základnými možnosťami ich vyriešenia.

Konkrétne sa pozrieme na hyperparametre (hlbokých) neurónových sietí, na vhodný výber ich hodnôt a na spôsoby urýchľovania trénovania hlbokých neurónových sietí, čo skoro vždy budete potrebovať, keďže výpočtové kapacity pre trénovanie sietí sú limitované.

## 1. Heuristika návrhu hlbokých sietí

| úloha                   | počet vstupov   | počet výstupov  | aktivačná funkcia vo výstupnej vrstve |
|-------------------------|-----------------|-----------------|---------------------------------------|
| binárna klasifikácia    | počet príznakov | 2               | sigmoid                               |
| multiclass klasifikácia | počet príznakov | C (počet tried) | softmax                               |
| predikcia               | počet príznakov | zvyčajne 1      | ReLU/linear                           |

Pri návrhu hlbokých sietí oplatí použiť veľa vrstiev iba v prípade konvolučných sietí, v plne prepojenej časti siete je odporúčané použiť max. 3 skryté vrstvy (ďalšie pridané skryté vrstvy nezvyšujú veľmi presnosť).

## 2. Trénovacie, testovacie a validačné množiny

Úspech, resp. neúspech trénovaných modelov závisí v prvom rade nielen od ich topológie a hyperparametrov, ale od použitých dát a ich rozdelenia. Najdôležitejším predpokladom pre úspešné nasadenie neurónových sietí je to, aby **príklady v trénovacích dátach zodpovedali očakávaným reálnym vstupným príkladom** pri používaní modelu na reálnu predikciu. Nesúlad môže spôsobiť problém napríklad pri trénovaní s obrázkami s vysokým rozlíšením a pri používaní na predikciu z obrázkov s nízkym rozlíšením a podobne.

Dáta v kontexte trénovania a testovania neurónových sietí zvyčajne rozdeľujeme na tri časti: **trénovacia** (*training*), **validačná** (*development* alebo *validation*) a **testovacia** (*test*) množina. Trénovacie dáta sa používajú na prispôsobovanie parametrov siete tak, aby predikcie siete boli čo najpresnejšie (fáza trénovania). Validačná množina slúži na vyhodnotenie presnosti siete ešte vo fáze trénovania, najmä na podporu rozhodnutia, či má ešte zmysel ďalej trénovať danú sieť. Testovacia množina napokon slúži na vyhodnotenie modelu z pohľadu používania - naším cieľom je nasimulovať čo najvhodnejšie prípady použitia.

V literatúre sa veľmi často stretnete s prípadom, kde sa použijú iba dve množiny, ktoré sú označené ako *trénovacia* a *testovacia* množina. Toto pomenúvanie je síce zaužívané, treba si ale uvedomiť, že *testovacia* množina v tomto kontexte skoro vždy hrá rolu *validačnej* množiny.

Základné odporúčanie na rozdelenie dostupných dát pre trénovanie neurónových sietí je *70/30%*, resp. *60/20/20%* v prípade rozdelenia do validačnej aj testovacej množiny. Toto rozdelenie slúži ako dobrá heuristika pre menšie datasety (desaťtisíce príkladov), avšak ak máte k dispozícii veľké dáta na trénovanie, priradenie *20-30%* príkladov do validačnej alebo testovacej množiny je často zbytočné.

V takýchto prípadoch treba brať do úvahy prvotný zámer použitia týchto množín - teda vyhodnotenie presnosti modelu, resp. porovnávanie dvoch modelov, a simulácia prípadov použitia. Práve preto ak máte státisíce alebo milióny údajov k dispozícii, veľmi často môžete priradiť až *90-98%* údajov do trénovacej množiny, a následne rozdeliť zvyšnú časť medzi validačnou a testovacou množinou (zvyčajne rovnomerne, alebo menej do testovacej).

### 2.1 Vyhodnotenie neurónovej siete

Spôsob vyhodnotenia natrénovaného modelu samozrejme závisí od konkrétneho príkladu použitia. Vo väčšine prípadov však vašou úlohou je dosiahnuť čo najvyššiu presnosť modelu a čo najmenšiu chybu. V iných prípadoch, kde cieľom je napríklad nahradiť existujúci proces, postačuje ak vaše riešenie prekonáva existujúce prístupy v istých metrikách.

Pri predikčných úlohách prvotnou metrikou je hodnota chyby, ktorú chcete znížiť. Pri klasifikácii správanie siete viete znázorniť pomocou konfúznej matice alebo kontingenčnej tabuľky. Konfúzna matica je tabuľková reprezentácia, kde v riadkoch máme očakávané triedy a v stĺpcoch vypočítané (predikované). V bunkách tabuľky sú uložené počty príkladov klasifikované v danej kombinácii očakávanej a predikovanej triedy. Ideálny klasifikátor bude mať všetky hodnoty po hlavnej diagonále (ďalšie informácie nájdete na [wikipédii](https://en.wikipedia.org/wiki/Confusion_matrix)).

Z konfúznej matice potom vieme vypočítať ďalšie metriky, ako správnosť (*accuracy*), návratnosť (*recall*) a presnosť (*precision*).

Správnosť popisuje samotný klasifikátor a vypočíta sa nasledovne:

$ACC = \frac{TP + TN}{P + N}$

kde $TP + TN$ je suma správne klasifikovaných príkladov (na hlavnej diagonále) a $P + N$ je počet všetkých príkladov.

Návratnosť (senzitivita) a presnosť popisujú klasifikátor pre danú triedu, vypočítajú sa nasledovne:

$REC = \frac{TP}{P}$

$PREC = \frac{TP}{TP + FP}$

kde $TP$ je počet správne klasifikovaných príkladov z danej triedy, $P$ je počet príkadov z danej triedy v testovacej množine a $FP$ je počet príkladov z testovacej množiny nesprávne klasifikovaných do tejto triedy.

Často sa vypočíta aj skóre F1, ktorá je harmonický priemer návratnosti a presnosti:

$F1 = 2 \cdot \frac{REC \cdot PREC}{REC + PREC}$

## 3. *Bias* a *variance*

Vývoj hlbokých modelov je iteratívny proces, pri ktorom navrhnete istú architektúru, vyhodnotíte ju, a na základe záveru vyhodnotenia prispôsobíte ho požiadavkám. V tomto smere vám pomôžu dva ukazovatele, a to *bias* a *variance* siete. *Bias* pritom vyjadruje schopnosť siete naučiť sa robiť presné predikcie, kým *variance* vyjadruje jej schopnosť generalizovať, t.j. zovšeobecniť predikciu na základe trénovacích dát. Tieto dva ukazovatele získate z číselných metrík - trénovacia (*training set error* - TSE) a validačná (*development set error* - DSE) chyba. Tieto chyby môžu byť vyjadrené ako priemerná chyba pri regresii, alebo percentuálna presnosť pri klasifikácii. Miera trénovacej chyby nám povie niečo o *biase*, kým porovnávanie trénovacej a validačnej chyby nám určí *variance*. Vysoký *bias* je dôsledkom podtrénovania, a vysoký *variance* zas pretrénovania.

Je dôležité povedať, že pri vývoji neurónových sietí by ste si mali určiť cieľovú presnosť, a rovnako uvažovať nad ľudskou presnosťou pre danú úlohu, resp. nad presnosťou náhodného klasifikátora. Pri vývoji sa najčastejšie stretnete s nasledovnými štyrmi prípadmi:

1. $TSE < DSE$, $TSE$ blízko alebo nižšia ako ľudská úroveň - váš model je pretrénovaný, nedokáže dobre generalizovať svoje znalosti na predikovanie; musíte skôr ukončiť trénovanie
2. $TSE \approx DSE$; nie sú blízko ľudskej úrovni - vysoký *bias*, váš model sa nedokáže natrénovať na trénovacích dátach; potrebujete viac údajov alebo vhodnejšie vybrať údaje, resp. predspracovať ich
3. $TSE>>, DSE>>$ - vysoký *bias* aj *variance*; váš model je pretrénovaný na niektoré časti údajov
4. $TSE<<, DSE<<$ - nízky *bias* aj *variance*; ideálny prípad.

Všeobecne vysoký *bias* riešite s robustnejšou topológiou, dlhším trénovaním, použitím iných optimalizačných algoritmov, alebo inou architektúrou siete. Vysoký *variance* sa rieši získaním väčšieho datasetu, regularizáciou údajov, a použitím inej architektúry.

## Použitá literatúra

* [Andrew Ng: Improving Deep Neural Networks - Hyperparameter Tuning](https://www.youtube.com/watch?v=1waHlpKiNyY&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc)